In [1]:
!pip install -r "/kaggle/input/requires/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:

token=""
from huggingface_hub import login
login(token=token)

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F
import os
import json
from tqdm import tqdm
import pickle




def last_token_pool(last_hidden_states, attention_mask):
    left_padding=(attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths=attention_mask.sum(dim=1) - 1
        batch_size=last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]




max_length=512


pretrained_model_name_or_path="Qwen/Qwen2-1.5B-Instruct"
which_embedding="Qwen2-1.5B-Instruct_embedding"

save_dir=f"/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)




print("Loading tokenizer...")
tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token

print("Loading model (this may take a few minutes)...")
model=AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path,
    trust_remote_code=True,
    device_map="auto",
)
model.eval()




def get_kl(model, input_texts):
    
    batch_dict=tokenizer(input_texts, max_length=max_length, padding=True, truncation=True, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs=model(**batch_dict, output_hidden_states=True)

        
        last_logits=model.lm_head(outputs.hidden_states[-1]).squeeze()
        first_logits=model.lm_head(outputs.hidden_states[0]).squeeze()

    kls=[]
    for i in range(1, len(outputs.hidden_states) - 1):
        with torch.no_grad():
            middle_logits=model.lm_head(outputs.hidden_states[i]).squeeze()
        kl_first=F.kl_div(
            F.log_softmax(middle_logits, dim=-1),
            F.softmax(first_logits, dim=-1),
            reduction="batchmean",
        )
        kl_last=F.kl_div(
            F.log_softmax(middle_logits, dim=-1),
            F.softmax(last_logits, dim=-1),
            reduction="batchmean",
        )
        kls.append((kl_first + kl_last).item())
    return kls

data_dir="/kaggle/input/bigmodel"






for root, dirs, files in os.walk(data_dir):  
    for file_name in files:
        if not file_name.endswith(".json"):
            continue  

        file_path=os.path.join(root, file_name)
        save_path=os.path.join(save_dir, file_name.split('.')[0] + '.pkl')

        if os.path.exists(save_path):
            print(f"Already exists: {save_path}")
            continue

        print(f"\nProcessing: {file_path}")
        with open(file_path, "r") as f:
            data=json.load(f)

        kls=[]
        for text_info in tqdm(data, desc=f"Computing KL for {file_name}"):
            text=text_info["text"]
            kl=get_kl(model, [text])
            kls.append(kl)
            if len(kls) >= 3000:  
                break

        print(f"Saving: {save_path}")
        with open(save_path, "wb") as f:
            pickle.dump(kls, f)

Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


Processing: /kaggle/input/bigmodel/HC3_en_test.json


Computing KL for HC3_en_test.json:   0%|          | 11/6526 [00:13<2:14:39,  1.24s/it]


KeyboardInterrupt: 

In [ ]:

from transformers import AutoTokenizer, AutoModel
import torch
import os
import json
from tqdm import tqdm
import torch.nn.functional as F




def last_token_pool(last_hidden_states, attention_mask):
    left_padding=(attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths=attention_mask.sum(dim=1) - 1
        batch_size=last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]




max_length=512


pretrained_model_name_or_path="Alibaba-NLP/gte-Qwen2-1.5B-instruct"
which_embedding="gte-qwen1.5-1.5B_all_embedding"




print("Loading tokenizer...")
tokenizer=AutoTokenizer.from_pretrained(pretrained_model_name_or_path, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token

print("Loading model (this may take a few minutes on first run)...")
model=AutoModel.from_pretrained(pretrained_model_name_or_path, trust_remote_code=True, device_map="auto")
model.eval()
def get_all_embedding(model, input_texts):
    batch_dict=tokenizer(
        input_texts,
        max_length=max_length,
        padding=True,
        truncation=True,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        outputs=model(
            **batch_dict,
            output_hidden_states=True,
            use_cache=False   
        )

    all_embed=[
        last_token_pool(outputs.hidden_states[i], batch_dict['attention_mask'])
        for i in range(len(outputs.hidden_states))
    ]
    all_embed=torch.cat(all_embed, dim=1).cpu()
    return all_embed




data_dir="/kaggle/input/bigmodel"
save_dir=f"/kaggle/working/"
os.makedirs(save_dir, exist_ok=True)




for file_name in os.listdir(data_dir):
    save_path=os.path.join(save_dir, file_name.split('.')[0] + '.pt')
    if os.path.exists(save_path):
        continue

    print(f"\nProcessing: {file_name}")
    with open(os.path.join(data_dir, file_name), "r") as f:
        data=json.load(f)

    embeddings=[]
    for text_info in tqdm(data, desc=f"Embedding {file_name}"):
        text=text_info["text"]
        embedding=get_all_embedding(model, [text])
        embeddings.append(embedding)
        if len(embeddings) >= 3000:  
            break

    embeddings=torch.cat(embeddings, dim=0)
    print("Embedding shape:", embeddings.shape)
    torch.save(embeddings, save_path)
    print("Saved:", save_path)


Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenization_qwen.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/gte-Qwen2-1.5B-instruct:
- tokenization_qwen.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Processing: HC3_en_test.json


Embedding HC3_en_test.json:  46%|████▌     | 2999/6526 [10:46<12:40,  4.64it/s]


Embedding shape: torch.Size([3000, 44544])
Saved: /kaggle/working/HC3_en_test.pt


In [ ]:




import os
import pickle
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm
from sklearn.metrics import (
    roc_auc_score, accuracy_score, f1_score,
    precision_score, recall_score, confusion_matrix
)


os.environ["CUDA_LAUNCH_BLOCKING"]="1"
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


EMB_BASE   ="/kaggle/input/newnewmodels"
KL_BASE    ="/kaggle/input/newnewpickle"
LABEL_BASE ="/kaggle/input/newlabels"

train_file="HC3_en_test"
valid_file="gpt4-Xsum-gpt3"
test_files=["gpt4-pub-gpt3", "gpt4-writing-gpt3"]

which_layer="max_kl"
learning_rate=1e-3
batch_size=16
num_epochs=10
hidden_sizes=[1024, 512]
dropout_prob=0.2

best_model_path="best_binary_classifier.pt"




def load_embedding(name):
    path=os.path.join(EMB_BASE, f"{name}.pt")
    print("Loading embedding:", path)
    return torch.load(path, map_location="cpu")

def load_labels(name):
    path=os.path.join(LABEL_BASE, f"{name}.pt")
    print("Loading labels:", path)
    labels=torch.load(path, map_location="cpu").long()

    if labels.min() < 0 or labels.max() > 1:
        raise ValueError(f"[ERROR] Labels for {name} contain non-binary values.")
    return labels

def load_kl(name):
    path=os.path.join(KL_BASE, f"{name}.pkl")
    print("Loading KL:", path)
    with open(path, "rb") as f:
        return pickle.load(f)




def infer_per_layer_dim(embed, sample_kl):
    L=len(sample_kl)
    total_states=L + 2
    D=embed.shape[1]

    if D % total_states != 0:
        raise ValueError(f"Dimension mismatch: D={D} states={total_states}")

    return D // total_states, total_states




def select_max_kl(embed, kl, per_layer):
    idx=kl.argmax(axis=1)
    selected=[]

    for i, layer in enumerate(idx):
        start=(layer + 1) * per_layer
        end  =(layer + 2) * per_layer
        selected.append(embed[i, start:end])

    return torch.stack(selected, dim=0)

def select_layer_embeddings(embed, kl, per_layer, mode):
    if mode == "max_kl":
        return select_max_kl(embed, kl, per_layer)
    elif mode == "last_layer":
        return embed[:, -per_layer:]
    else:
        raise ValueError("Unknown which_layer:", mode)




class BinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, dropout_prob):
        super().__init__()

        layers=[]
        prev=input_size

        for h in hidden_sizes:
            layers += [
                nn.Dropout(dropout_prob),
                nn.Linear(prev, h),
                nn.ReLU(),
            ]
            prev=h

        self.dense=nn.Sequential(*layers)
        self.classifier=nn.Linear(prev, 2)

    def forward(self, x):
        return self.classifier(self.dense(x))




def prepare_dataset(name, max_items=None):
    emb=load_embedding(name)
    labels=load_labels(name)
    kl=np.array(load_kl(name))

    min_size=min(len(emb), len(labels), len(kl))
    if max_items is not None:
        min_size=min(min_size, max_items)

    emb=emb[:min_size]
    labels=labels[:min_size]
    kl=kl[:min_size]

    emb_sel=select_layer_embeddings(
        embed=emb,
        kl=kl,
        per_layer=per_layer_dim,
        mode=which_layer,
    )

    return emb_sel.to(device), labels.to(device)




print("\nInferring layer dims from training file…")
sample_embed=load_embedding(train_file)
sample_kl=np.array(load_kl(train_file))

per_layer_dim, total_states=infer_per_layer_dim(sample_embed, sample_kl[0])
print("Per-layer hidden dim:", per_layer_dim)
print("Total states:", total_states)




print("\nPreparing datasets...")
train_embeddings, train_labels=prepare_dataset(train_file, max_items=3000)
valid_embeddings, valid_labels=prepare_dataset(valid_file, max_items=3000)

test_embeddings=[]
test_labels=[]
for t in test_files:
    try:
        e, l=prepare_dataset(t, max_items=3000)
        test_embeddings.append(e)
        test_labels.append(l)
    except Exception as ex:
        print("Skipping:", t, "| Reason:", ex)




def evaluate(model, emb, labels):
    model.eval()
    with torch.no_grad():
        logits=model(emb)
        probs=torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        preds=(probs >= 0.5).astype(int)
        labels_np=labels.cpu().numpy()

        return {
            "acc": accuracy_score(labels_np, preds),
            "auroc": roc_auc_score(labels_np, probs),
            "precision": precision_score(labels_np, preds, zero_division=0),
            "recall": recall_score(labels_np, preds, zero_division=0),
            "f1": f1_score(labels_np, preds, zero_division=0),
            "confusion_matrix": confusion_matrix(labels_np, preds).tolist(),
        }




def train_model():
    model=BinaryClassifier(
        input_size=train_embeddings.shape[1],
        hidden_sizes=hidden_sizes,
        dropout_prob=dropout_prob
    ).to(device)

    opt=torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn=nn.CrossEntropyLoss()

    best_acc=-1

    for epoch in range(num_epochs):
        model.train()
        perm=torch.randperm(train_embeddings.shape[0])

        total_loss=0

        for i in range(0, len(perm), batch_size):
            idx=perm[i:i+batch_size]
            emb=train_embeddings[idx]
            lab=train_labels[idx]

            opt.zero_grad()
            loss=loss_fn(model(emb), lab)
            loss.backward()
            opt.step()

            total_loss += loss.item()

        val=evaluate(model, valid_embeddings, valid_labels)

        print(
            f"Epoch {epoch+1:02d} | loss={total_loss:.4f} | "
            f"acc={val['acc']:.4f} | f1={val['f1']:.4f} | "
            f"precision={val['precision']:.4f} | recall={val['recall']:.4f} | "
            f"auroc={val['auroc']:.4f}"
        )

        if val["acc"] > best_acc:
            best_acc=val["acc"]
            torch.save(model.state_dict(), best_model_path)
            print(" Saved BEST model.")

    print("\nTraining complete. Best validation acc:", best_acc)




train_model()
print("\nDONE.")



Inferring layer dims from training file…
Loading embedding: /kaggle/input/newnewmodels\HC3_en_test.pt


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/newnewmodels\\HC3_en_test.pt'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    confusion_matrix, roc_auc_score, f1_score
)
import numpy as np





X_train=train_embeddings.cpu().numpy()
y_train=train_labels.cpu().numpy()

X_valid=valid_embeddings.cpu().numpy()
y_valid=valid_labels.cpu().numpy()

test_sets_np=[]
for e, l in zip(test_embeddings, test_labels):
    test_sets_np.append((e.cpu().numpy(), l.cpu().numpy()))


In [ ]:
def evaluate_classifier(model, X, y):
    
    probs=model.predict_proba(X)[:, 1]
    preds=(probs >= 0.5).astype(int)

    return {
        "acc": accuracy_score(y, preds),
        "auroc": roc_auc_score(y, probs),
        "precision": precision_score(y, preds, zero_division=0),
        "recall": recall_score(y, preds, zero_division=0),
        "f1": f1_score(y, preds, zero_division=0),
        "confusion_matrix": confusion_matrix(y, preds).tolist(),
    }


In [ ]:
print("\n======================")
print("Training Random Forest")
print("======================")

rf=RandomForestClassifier(
    n_estimators=600,
    max_depth=40,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1,
)

rf.fit(X_train, y_train)
print(" Random Forest trained!")

print("\n=== RF Validation ===")
rf_val=evaluate_classifier(rf, X_valid, y_valid)
for k, v in rf_val.items():
    print(f"{k}: {v}")


for (X, y), name in zip(test_sets_np, test_files):
    print(f"\n=== RF Test: {name} ===")
    r=evaluate_classifier(rf, X, y)
    for k, v in r.items():
        print(f"{k}: {v}")



Training Random Forest


NameError: name 'RandomForestClassifier' is not defined

In [ ]:
print("\n================")
print("Training SVM RBF")
print("================")

svm=SVC(
    kernel="rbf",
    C=15,
    gamma="scale",
    probability=True,   
    class_weight="balanced",
    random_state=42
)

svm.fit(X_train, y_train)
print(" SVM trained!")

print("\n=== SVM Validation ===")
svm_val=evaluate_classifier(svm, X_valid, y_valid)
for k, v in svm_val.items():
    print(f"{k}: {v}")


for (X, y), name in zip(test_sets_np, test_files):
    print(f"\n=== SVM Test: {name} ===")
    r=evaluate_classifier(svm, X, y)
    for k, v in r.items():
        print(f"{k}: {v}")



Training SVM RBF


NameError: name 'SVC' is not defined

In [ ]:




print("\nLoading best binary classifier...")
best_model=BinaryClassifier(
    input_size=train_embeddings.shape[1],
    hidden_sizes=hidden_sizes,
    dropout_prob=dropout_prob
).to(device)

best_model.load_state_dict(torch.load(best_model_path, map_location=device))
best_model.eval()

print("\n---- VALIDATION SET METRICS ----")
val_metrics=evaluate(best_model, valid_embeddings, valid_labels)
print(val_metrics)


print("\n---- TEST SET METRICS ----")
for emb, lab in zip(test_embeddings, test_labels):
    metrics=evaluate(best_model, emb, lab)
    print(metrics)



Loading best binary classifier...

---- VALIDATION SET METRICS ----
{'acc': 0.825503355704698, 'auroc': 0.8942389982433224, 'precision': 0.8702290076335878, 'recall': 0.7651006711409396, 'f1': 0.8142857142857143, 'confusion_matrix': [[132, 17], [35, 114]]}

---- TEST SET METRICS ----
{'acc': 0.5503355704697986, 'auroc': 0.573983153911986, 'precision': 0.6271186440677966, 'recall': 0.2483221476510067, 'f1': 0.3557692307692308, 'confusion_matrix': [[127, 22], [112, 37]]}
{'acc': 0.77, 'auroc': 0.8619111111111112, 'precision': 0.8403361344537815, 'recall': 0.6666666666666666, 'f1': 0.7434944237918216, 'confusion_matrix': [[131, 19], [50, 100]]}
